<h2>Conectando no mysql</h2>

In [ ]:
# Pegando a senha para o mysql

with open("access/password_sql.txt","r") as f:
    _pass_mysql = f.read()

In [ ]:
import mysql.connector
from mysql.connector import errorcode

try:
    banco = mysql.connector.connect(
        host = "localhost",
        user = "root",
        password = _pass_mysql,
        database = "marketing",
        auth_plugin = 'mysql_native_password'
    )
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Cheque sua senha ou usuario, identificado um erro em alguma delas!")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("""'{}' esse database não foi encontrado, verifique se digitou corretamente.""".format(banco.database))
    else:
        print(err)
    
cursor = banco.cursor()

### Declarando as variaveis para a requisição da **API**

In [ ]:
# pegando o token de acesso para o fb
with open("access/access_facebook.txt","r") as f:
    _data = f.read().split("\n")
    _access_token = str(_data[0].split("=")[1])
    _n_ad = str(_data[1].split("=")[1])

In [ ]:
# variaveis para instanciar a classe
token_acesso = _access_token
numero_conta = _n_ad

# variaveis para chamar a função do tipo de requisição
level_breakdown = "ad"

<h3>Declarando a classe de requisição e bibliotecas necessarias</h3>

In [ ]:
import requests
import json
import time
from datetime import date
import datetime
import csv
import pandas as pd
import warnings

class requisicao:
    # inicio o objeto de requisicao pegando as info principais
    def __init__(self,n_account,token):
        self.n_account = n_account
        self.conta_ad = "act_" + n_account
        self.token = token
        self.url = "https://graph.facebook.com/v12.0/{}/insights".format(self.conta_ad)
        self.limit = '2000'
        
    def mostrar_req(self):
        print("----------- mostrar_req() -------------")
        print("Nº da conta de anúncio: {}".format(self.conta_ad))
        print("URL padrão: {}".format(self.url))
        print("----------------------------------------")

    def requisicaoPadrao(self,level,data):
        """Realiza o resquest a API de marketing do facebook, usando os campos padrões, deixando as ações de lado. Formato data YYYY-MM-DD"""
        fields = "campaign_name,adset_name,ad_name,spend,impressions,clicks,ad_id"
        time_range = "{{\"since\":\"{}\",\"until\":\"{}\"}}".format(data,data)
        parametros = {'time_range':time_range,'level':level,'fields':fields,'limit':self.limit,'access_token':self.token}
        r = requests.get(self.url,params=parametros)
        return r

    def requisicaoActions(self,level,data):
        """Realiza o resquest a API de marketing do facebook, usando somente o campo de action e ad_id. Formato data YYYY-MM-DD"""
        fields = "ad_id,actions"
        time_range = "{{\"since\":\"{}\",\"until\":\"{}\"}}".format(data,data)
        parametros = {'time_range':time_range,'level':level,'fields':fields,'limit':self.limit,'access_token':self.token}
        r = requests.get(self.url,params=parametros)
        return r

### Instanciando a classe *requisicao*

In [ ]:
obj_req = requisicao(numero_conta,token_acesso)

obj_req.mostrar_req()

### Lendo o banco de dados, checando os dias

In [ ]:
# checar os dias PADRAO
for ite_dia in range(20,0,-1):
    check_dia = date.today() - datetime.timedelta(days=ite_dia)
    cursor.execute("SELECT * FROM marketing.fb_ads_data_padrao WHERE dia = '{}'".format(check_dia))
    contagem = 0
    for i in cursor:
        contagem += 1
    if contagem > 0:
        continue
    elif contagem == 0:
        call_req = obj_req.requisicaoPadrao(level_breakdown,check_dia)
        for campanha in call_req.json()['data']:
            #aqui tenho cada ad_id, passar as linhas na tabela
            adicionar_linha = (
                "INSERT INTO marketing.fb_ads_data_padrao"
                "(dia, ad_id, campaign_name, adset_name, ad_name, spend, impressions, clicks)"
                "VALUES ('{}','{}','{}','{}','{}',{},{},{})".format(check_dia,campanha['ad_id'],campanha['campaign_name'],campanha['adset_name'],campanha['ad_name'],campanha['spend'],campanha['impressions'],campanha['clicks'])
            )
            print("PADRAO")
            print(adicionar_linha)
            print()
            cursor.execute(adicionar_linha)
            emp_no = cursor.lastrowid
            banco.commit()
        time.sleep(2)

# checar os dias ACTIONS
dic_ad_actions = {
    "comment":0,
    "onsite_conversion.post_save":0,
    "post_engagement":0,
    "post_reaction":0,
    "link_click":0,
    "landing_page_view":0,
    "initiate_checkout":0,
    "view_content":0,
    "purchase":0,
    "lead":0
}

for ite_dia in range(20,0,-1):
    check_dia = date.today() - datetime.timedelta(days=ite_dia)
    cursor.execute("SELECT * FROM marketing.fb_ads_data_actions WHERE dia = '{}'".format(check_dia))
    contagem = 0
    for i in cursor:
        contagem += 1
    if contagem > 0:
        continue
    elif contagem == 0:
        # fazer a requisição
        call_req = obj_req.requisicaoActions(level_breakdown,check_dia)
        for usar in call_req.json()['data']:
            
            #aqui tenho cada ad_id, passar as linhas na tabela
            if 'actions' not in usar:
                adicionar_linha = (
                    "insert into marketing.fb_ads_data_actions"
                    "(dia, ad_id, post_comment, post_save, post_engagement, post_reaction, link_click, lp_view, website_checkout, content_view, purchase, leads)"
                    "values (\'{}\',\'{}\',{},{},{},{},{},{},{},{},{},{})".format(check_dia,usar['ad_id'],0,0,0,0,0,0,0,0,0,0)
                )
                cursor.execute(adicionar_linha)
                emp_no = cursor.lastrowid
                banco.commit()
                continue
            for ite_tipo_action in usar['actions']:
                if ite_tipo_action['action_type'] in dic_ad_actions:  
                    dic_ad_actions[ite_tipo_action['action_type']] = ite_tipo_action['value']
                    
            adicionar_linha = (
                "insert into marketing.fb_ads_data_actions"
                "(dia, ad_id, post_comment, post_save, post_engagement, post_reaction, link_click, lp_view, website_checkout, content_view, purchase, leads)"
                "values (\'{}\',\'{}\',{},{},{},{},{},{},{},{},{},{})".format(check_dia,usar['ad_id'],dic_ad_actions['comment'],dic_ad_actions['onsite_conversion.post_save'],dic_ad_actions['post_engagement'],dic_ad_actions['post_reaction'],dic_ad_actions["link_click"],dic_ad_actions['landing_page_view'],dic_ad_actions['initiate_checkout'],dic_ad_actions['view_content'],dic_ad_actions['purchase'],dic_ad_actions['lead'])
            )
            print("ACTIONS")
            print(adicionar_linha)
            print()
            # limpar o dicionario
            for valor_dic in dic_ad_actions:
                dic_ad_actions[valor_dic] = 0
                
            cursor.execute(adicionar_linha)
            emp_no = cursor.lastrowid
            banco.commit()
        time.sleep(2)

In [ ]:
# finalizando a aplicação, executar quando acabar.
cursor.close()
banco.close()

<h3>Criando o database</h3>

In [ ]:
# criando listas com as linhas que vamos executar no MySQL

database_name = "marketing"

database_linhas_iniciais = [f"create database {database_name}"]

def create_database(cursor):
    """Executar o cursor.execute() com as linhas de comando iniciais para criação de nosso database"""
    try:
        for database_linha in database_linhas_iniciais:
            cursor.execute(database_linha)
    except mysql.connector.Error as err:
        print("""Não foi possivel criar o database '{database_name}': {err}""".format(database_name,err))
        exit(1)
        # (0) --> significa uma saida limpa, sem um erro que causou a saida.
        # (1) --> significa que houve um erro, por causa do erro a aplicação vai encerrar.

try:
    cursor.execute("USE {}".format(database_name))
except mysql.connector.Error as err:
    print("""Database '{}' não foi encontrado""".format(database_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("""Database '{}' foi criado com sucesso""".format(database_name))
        cursor.database = database_name
    else:
        print(err)
        exit(1)

<h3>Rodando os comandos para criação da tabela dentro do database</h3>

In [ ]:
for table in tabelas:
    table_description = tabelas[table]
    try:
        print("""Criando tabela no database: '{}' |  """.format(database_name), end = "")
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("""Tabela '{}' já existe""".format(table))
        else:
            print(err.msg)
    else:
        print("Tudo certo, champs :D")
